In [4]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [2]:
# Load California Housing dataset
data = fetch_california_housing()
X = data.data
y = data.target.reshape(-1, 1)

# Standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42)

In [3]:
# Custom loss: Weighted MSE (example: more weight to higher values)
def custom_weighted_mse(y_true, y_pred):
    weights = tf.where(y_true > 2.0, 2.0, 1.0)
    loss = tf.reduce_mean(tf.square(y_true - y_pred) * weights)
    return loss


In [5]:
model = Sequential([
    Dense(64, activation='relu', input_shape=(X.shape[1],)),
    Dense(1)
])

model.compile(optimizer='adam', loss=custom_weighted_mse, metrics=[])

c:\Users\Admin\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [6]:
# Train for 10 epochs
history = model.fit(X_train, y_train, epochs=10, verbose=1)


Epoch 1/10
452/452 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 3.6628
Epoch 2/10
452/452 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.8142
Epoch 3/10
452/452 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.6835
Epoch 4/10
452/452 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.6725
Epoch 5/10
452/452 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.6307
Epoch 6/10
452/452 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.6371
Epoch 7/10
452/452 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.6348
Epoch 8/10
452/452 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.6280
Epoch 9/10
452/452 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.5981
Epoch 10/10
452/452 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.5747


In [7]:
# Predict on test set
y_pred = model.predict(X_test)

# Compute standard MSE
standard_mse = mean_squared_error(y_test, y_pred)

# Compute custom weighted MSE on test set
y_true_tensor = tf.convert_to_tensor(y_test, dtype=tf.float32)
y_pred_tensor = tf.convert_to_tensor(y_pred, dtype=tf.float32)
custom_mse = custom_weighted_mse(y_true_tensor, y_pred_tensor).numpy()

194/194 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


In [8]:
# Print comparison table
print("\nComparison of Loss Metrics on Test Data")
print("----------------------------------------")
print(f"{'Standard MSE':<20}: {standard_mse:.4f}")
print(f"{'Custom Weighted MSE':<20}: {custom_mse:.4f}")



Comparison of Loss Metrics on Test Data
----------------------------------------
Standard MSE        : 0.3743
Custom Weighted MSE : 0.5882
